In [1]:
import numpy as np
import pickle
from random import choices
import pandas as pd
from dataset_helpers import *
import os
import torch
save_path = os.getcwd() + '/output/'

# Structures

In [ ]:
with open(os.getcwd() + '/houses/training_data.pkl','rb') as f:
    training_data = pickle.load(f)
    
with open(os.getcwd() + '/houses/validation_data.pkl','rb') as f:
    validation_data = pickle.load(f)

data = []
for i in range(len(training_data)):
    data.append((training_data[i][1], training_data[i][2]))
for i in range(len(validation_data)):
    data.append((validation_data[i][1], validation_data[i][2]))

In [ ]:
# testing the annotation reversal
a = [[[1,1,2], [3,4,5], [6,7,8]]]
b = ['nothing', 'roof', 'windows', 'windows', 'windows', 'windows', 'windows', 'none', 'wall' ]

reversed_annotation = b[1:] # exclude 'nothing' from re-indexing
reversed_annotation.reverse()
resorted_annotation = ['nothing']
resorted_annotation += reversed_annotation

print(clean_segmentation(np.array(a), b))
print(resorted_annotation)

[[[8 8 0]
  [0 0 0]
  [0 0 1]]]
['nothing', 'wall', 'none', 'windows', 'windows', 'windows', 'windows', 'windows', 'roof']


In [ ]:
structures = []
for (segmentation, annotation) in data:
    structure = clean_segmentation(segmentation, annotation)
    reversed_annotation = annotation[1:] # exclude 'nothing' tag from reversal
    reversed_annotation.reverse()
    resorted_annotation = ['nothing']
    resorted_annotation += reversed_annotation
    structures.append((structure, resorted_annotation))
    for n in range(1, 4):
        structures.append((np.rot90(structure, k=n, axes=(0, 2)), resorted_annotation))

In [ ]:
with open(save_path + 'structures.pkl', 'wb') as f:
    pickle.dump(structures, f, protocol=pickle.DEFAULT_PROTOCOL)

# Markov Chains

In [ ]:
with open(save_path + 'structures.pkl','rb') as f:
    structures = pickle.load(f)

In [ ]:
transition_table, segments_dict = calculate_markov_transitions(structures)

In [ ]:
with open(save_path + 'transition_table.pkl', 'wb') as f:
    pickle.dump(transition_table, f, protocol=pickle.DEFAULT_PROTOCOL)

with open(save_path + 'segments_dict.pkl', 'wb') as f:
    pickle.dump(segments_dict, f, protocol=pickle.DEFAULT_PROTOCOL)

# Data Generation

In [2]:
with open(save_path + 'transition_table.pkl','rb') as f:
    transition_table = pickle.load(f)

with open(save_path + 'segments_dict.pkl','rb') as f:
    segments_dict = pickle.load(f)

In [3]:
DIM = 16
down = ['floor', 'base', 'bottom', 'ground', 'foundation']
up = ['roof', 'top', 'layer', 'ceiling', 'ledge', 'overhang', 'platform']

# create ids for segment
label_ids = {k: i for i, k in enumerate(segments_dict.keys())}

def get_y_indeces(structure, x, z, segment, name):
    s0, s1, s2 = segment.shape

    for word in down:
        # floor types go to the bottom
        if name.find(word) != -1:
            return 0, s1

    for word in up:
        # roof types go to the top
        if name.find(word) != -1:
            # roofs must not float
            # find highest point in designated area
            non_zero_ids = np.nonzero(
                # slice the requested area
                structure[x:x+s0, 0:DIM, z:z+s2])
            # use the highest non-zero y index
            if len(non_zero_ids[1]) == 0:
                # roof part falls to the ground
                return 1, 1+s1
            # prevent building from clipping OOB vertically
            h = non_zero_ids[1].max()
            safety_clip = h + s1 - DIM if h + s1 >= DIM else 0
            return (h - safety_clip, h + s1 - safety_clip)

    # wall types stand upright on the floor
    return 1, 1+s1

def generate_structure(annotation, segments_dict):
    structure = np.zeros(shape=(DIM,DIM,DIM), dtype=np.uint8)
    # start building from the outer thirds inward
    outer_thirds = [i for i in range(int(DIM/3))] + [i for i in range(int(2*DIM/3), DIM)]
    [x, z] = choices(outer_thirds, k=2)
    y = 0 # vertical dim in minetest

    # in the outer thirds of the space we invert horizontal directions
    turn_positive_zone = list(range(int(DIM/3)))
    turn_negative_zone = [2 * int(DIM/3) + i for i in range(int(DIM/3))]
    x_dir = 1 if x in turn_positive_zone else -1
    z_dir = 1 if z in turn_positive_zone else -1

    sequence = []

    for segment_index, segment_label in enumerate(annotation):
        if segment_index == 0:
            continue
        if segment_label == "Done":
            break
        choice, segment_uid = choices(segments_dict[segment_label])[0]
        # prevent alteration of the segment dict
        segment = np.copy(choice)
        segment[segment == 1] = segment_index
        s0, s1, s2 = segment.shape

        x_dir = 1 if x in turn_positive_zone else x_dir
        z_dir = 1 if z in turn_positive_zone else z_dir
        x_dir = -1 if x in turn_negative_zone else x_dir
        z_dir = -1 if z in turn_negative_zone else z_dir
        # some safety constraints
        x_dir *= -1 if x+x_dir*s0 >= DIM or x+x_dir*s0 < 0 else 1
        z_dir *= -1 if z+z_dir*s2 >= DIM or z+z_dir*s2 < 0 else 1
        next_x = x+x_dir*s0
        next_z = z+z_dir*s2
        
        # update vertical position according to segment type
        y, next_y = get_y_indeces(structure, x, z, segment, segment_label)
        assert(y < next_y)
        
        # clip segments which do not fit in either direction
        diff = [0, 0, 0]
        for i, s in enumerate(structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir].shape):
            diff[i] = s - segment.shape[i]
        if diff != [0, 0, 0]:
            segment = segment[0:s0+diff[0], 0:s1+diff[1], 0:s2+diff[2]]
        
        segment_encoding = [
            label_ids[segment_label],
            x, next_x, x_dir, y, next_y, z, next_z, z_dir
            ]
        sequence.append(segment_encoding)

        try:
            structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir] = segment
        except Exception as e:
            print('space:', structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir].shape)
            print('segment:', segment.shape)
            print('diff:', diff)

        # build along the same axis
        if s0 > s2:
            x = next_x
        else:
            z = next_z
    
    # position the structure randomly to make it more easily
    # distinguishable from noise than dense structures
    # padded_structure = np.zeros((24, 24, 24), dtype=np.uint8)
    # [rx, rz] = choices(list(range(6)), k=2)
    # padded_structure[rx:rx+DIM, 0:+DIM, rz:rz+DIM] = structure
    return structure, np.asarray(sequence, dtype=np.int16)

# testing that everything works as expected
markov_annotation = generate_annotation(transition_table, 20)
print(markov_annotation)
print(len(markov_annotation))
artificial_structure, sequence = generate_structure(markov_annotation, segments_dict)
print(artificial_structure.shape)
print(sequence.shape)

['Start', 'floor', 'wall', 'floor', 'floor', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'roof', 'Done']
28
(16, 16, 16)
(26, 9)


In [4]:
from tqdm import tqdm
segmentation_only = []
segmentation_and_sequence = []
for i in tqdm(range(100000)):
    annotation = generate_annotation(transition_table)
    structure, sequence = generate_structure(annotation, segments_dict)
    segmentation_only.append(structure)
    segmentation_and_sequence.append((structure, sequence))

100%|██████████| 100000/100000 [02:39<00:00, 627.35it/s]


In [7]:
print(len(segmentation_and_sequence))
print(segmentation_and_sequence[0][0].dtype)
print(segmentation_and_sequence[0][0].shape)
print(segmentation_and_sequence[0][1].dtype)
print(segmentation_and_sequence[0][1].shape)

100000
uint8
(16, 16, 16)
int16
(21, 9)


In [6]:
segmentation_only = np.asarray(segmentation_only, dtype=np.float32) / 32.0
segmentation_only = torch.from_numpy(segmentation_only)

print(segmentation_only.size())
print(segmentation_only.dtype)
print(segmentation_only.min())
print(segmentation_only.max())

torch.Size([100000, 16, 16, 16])
torch.float32
tensor(0.)
tensor(1.)


In [ ]:
with open(save_path + 'segmentation_only.pkl', 'wb') as f:
    pickle.dump(segmentation_only, f, protocol=pickle.DEFAULT_PROTOCOL)

In [ ]:
with open(save_path + 'segmentation_and_sequence.pkl', 'wb') as f:
    pickle.dump(segmentation_and_sequence, f, protocol=pickle.DEFAULT_PROTOCOL)